# Threading
threading module provides a high-level interface for creating and managing threads, enabling concurrent execution of code. This is particularly useful for I/O-bound tasks, such as file operations or network communication.

Creating and Starting Threads

To create a thread, instantiate a Thread object from the threading module and specify the target function to execute:

In [1]:
import threading

def worker():
    print("Thread is running")

# Create a Thread object
thread = threading.Thread(target=worker)

# Start the thread
thread.start()

# Wait for the thread to complete
thread.join()


Thread is running


### Daemon Threads

Threads can be set as daemon threads, which means they will automatically exit when the main program exits:



In [ ]:
import threading
import time

def daemon_worker():
    while True:
        print("Daemon thread is running")
        time.sleep(1)

# Create a daemon thread
daemon_thread = threading.Thread(target=daemon_worker)
daemon_thread.daemon = True
daemon_thread.start()

# Main program sleeps for 3 seconds
time.sleep(3)
print("Main program exiting")


### Synchronizing Threads

When multiple threads access shared resources, synchronization mechanisms like locks are essential to prevent race conditions:



In [4]:
import threading

# Shared resource
counter = 0
counter_lock = threading.Lock()

def increment_counter():
    global counter
    for _ in range(1000):
        with counter_lock:
            counter += 1

# Create multiple threads
threads = []
for _ in range(10):
    thread = threading.Thread(target=increment_counter)
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

print(f"Final counter value: {counter}")


Final counter value: 10000


### Thread Communication with Queues

The queue module provides a thread-safe way for threads to communicate:

In [ ]:
import threading
import queue

def producer(q):
    for i in range(5):
        q.put(i)
        print(f"Produced {i}")

def consumer(q):
    while True:
        item = q.get()
        if item is None:
            break
        print(f"Consumed {item}")
        q.task_done()

q = queue.Queue()
producer_thread = threading.Thread(target=producer, args=(q,))
consumer_thread = threading.Thread(target=consumer, args=(q,))

producer_thread.start()
consumer_thread.start()

producer_thread.join()
q.put(None)  # Signal the consumer to exit
consumer_thread.join()


### Thread Pools with concurrent.futures

For managing a pool of threads, the ThreadPoolExecutor from the concurrent.futures module is convenient:

In [6]:
from concurrent.futures import ThreadPoolExecutor

def task(n):
    print(f"Processing {n}")

with ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(task, range(5))


Processing 0
Processing 1
Processing 2
Processing 3
Processing 4


### Deadlock

A deadlock occurs when two or more threads are blocked forever, each waiting for the other to release a resource. This situation halts the progress of the involved threads and can freeze the entire program.

Example of Deadlock:

In [7]:
import threading

# Initialize two locks
lock_a = threading.Lock()
lock_b = threading.Lock()

def thread1():
    with lock_a:
        print("Thread 1 acquired lock_a")
        with lock_b:
            print("Thread 1 acquired lock_b")

def thread2():
    with lock_b:
        print("Thread 2 acquired lock_b")
        with lock_a:
            print("Thread 2 acquired lock_a")

# Create and start threads
t1 = threading.Thread(target=thread1)
t2 = threading.Thread(target=thread2)
t1.start()
t2.start()
t1.join()
t2.join()


Thread 1 acquired lock_a
Thread 1 acquired lock_b
Thread 2 acquired lock_b
Thread 2 acquired lock_a


### Starvation

Starvation happens when a thread is perpetually denied access to resources, preventing it from making progress. This can occur if other threads monopolize the resources, leaving the starved thread waiting indefinitely.

Example of Starvation:




In [ ]:
import threading
import time

# Shared resource
resource_lock = threading.Lock()

def greedy_thread():
    while True:
        with resource_lock:
            print("Greedy thread acquired the resource")
            time.sleep(0.1)  # Holds the resource for a short time

def polite_thread():
    while True:
        with resource_lock:
            print("Polite thread acquired the resource")
            time.sleep(0.1)  # Holds the resource for a short time
        time.sleep(0.1)  # Allows other threads a chance to acquire the resource

# Create and start threads
t1 = threading.Thread(target=greedy_thread)
t2 = threading.Thread(target=polite_thread)
t1.start()
t2.start()


### Global Interpreter Lock (GIL)

The Global Interpreter Lock (GIL) is a mutex in CPython that allows only one thread to execute Python bytecode at a time. This design simplifies memory management but can limit the effectiveness of CPU-bound multithreaded programs.

Implications of the GIL:

I/O-Bound Tasks: For tasks involving input/output operations, such as file handling or network communication, the GIL is less of a bottleneck. While one thread waits for I/O operations to complete, the GIL can be released, allowing other threads to run.

CPU-Bound Tasks: In CPU-intensive operations, the GIL can hinder performance gains from multithreading, as only one thread executes at a time. In such cases, multiprocessing or implementing performance-critical sections in C extensions can be more effective.

Example Illustrating the GIL:




In [8]:
import threading

def cpu_bound_task():
    count = 0
    for _ in range(10**6):
        count += 1
    print("Task completed")

# Create and start multiple threads
threads = [threading.Thread(target=cpu_bound_task) for _ in range(4)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()


Task completed
Task completed
Task completed
Task completed


### To avoid GIL
To effectively bypass Python's Global Interpreter Lock (GIL) and achieve parallelism, especially for CPU-bound tasks, you can utilize the multiprocessing module. This approach allows you to run separate processes, each with its own Python interpreter and memory space, thereby sidestepping the GIL's limitations.

Example: Using the multiprocessing Module

The following example demonstrates how to use the multiprocessing module to perform parallel processing:

In [ ]:
from multiprocessing import Process

def cpu_bound_task():
    count = 0
    for _ in range(10**6):
        count += 1
    print("Task completed")

if __name__ == '__main__':
    # Create and start multiple processes
    processes = [Process(target=cpu_bound_task) for _ in range(4)]
    for process in processes:
        process.start()
    for process in processes:
        process.join()
